In [12]:
import ujson

In [20]:
# input paths
tfidf_path = './output/tfidf_results_new.jsonl'
w2v_path = './output/word2vec_results_new.jsonl'
bm25_path = './output/bm25_results_new.jsonl'
combo_path = './output/combined_results_new.jsonl'

# output paths
tfidf_selected_path = './output/tfidf_selected.jsonl'
w2v_selected_path = './output/w2v_selected.jsonl'
bm25_selected_path = './output/bm25_selected.jsonl'
combo_selected_path = './output/combo_selected_path.jsonl'

In [19]:
# TF-IDF
with open(tfidf_path) as f:
    for line in f:
        obj = ujson.loads(line)

        print(sorted(obj['tfidf_sim_scores'], key=lambda x: x['score'], reverse=True))
        break


[{'score': 0.0548429083, 'sentence_index': 1299947}, {'score': 0.0221197567, 'sentence_index': 2719612}]


In [15]:
# W2V
with open(w2v_path) as f:
    for line in f:
        obj = ujson.loads(line)
        t += len(obj['w2v_results'])
        for res in obj['w2v_results']:
            if res['score'] == 'nan':
                c += 1
print(c)
print(t)

286530
790708


In [ ]:
with open(bm25_path) as f:
    for line in f:
        obj = ujson.loads(line)
        for res in obj['bm25_results']:
            pass

In [ ]:
with open(combo_path) as f:
    for line in f:
        obj = ujson.loads(line)
        for res in obj['combination_results']:
            pass